## 웹 크롤링

### 인터넷 접속 라이브러리 추가

- urllib.request 모듈로 작업
- html 소스 로드로 별다른 문제 없음


In [22]:
from urllib.request import urlopen, Request

# 도시 별 날씨 검색 함수
def get_weather(city):
    # 기상청 홈페이지 도시별 날씨 페이지
    url = 'https://www.weather.go.kr/w/obs-climate/land/city-obs.do'
    page = urlopen(url=url)
    
    text = page.read().decode('utf-8')
    text = text[text.find(f'>{city}</a>'):]

    # 기온 가져오기
    for i in range(7):
        text = text[text.find('<td>')+1:]
  
    start = 3     # td> 이후 시작점 설정
    end = text.find('</td>')
    current_temp = text[start:end]
    print(f'{city}의 현재 기온은 {current_temp}˚C입니다.')
    
    # 습도 가져오기
    for i in range(3):
            text = text[text.find('<td>')+1:]
    
    start = 3
    end = text.find('</td>')
    current_humid = text[start:end] 
    print(f'{city}의 현재 기온은 {current_humid}%입니다.')
get_weather('부산')    


부산의 현재 기온은 11.6˚C입니다.
부산의 현재 기온은 26%입니다.


### OpenAPI 크롤링

- 공공데이터 포털
    - https://www.data.go.kr
    - 부산광역시 버스정보안내기 현황

- 문제점
    
```python
response = requests.get(total_url, verify = False)  # line 45
```
- 문제점
    - https를 일반 request로 부르면 SSL 오류 발생
    - 해결하려면 외부모듈 requests를 사용, verify=False 옵션을 지정



In [23]:
import requests
from urllib.parse import quote, unquote, urlencode   # 한글을 urlencode로 변환 해줌  롯데 ->'%EB%A1%AF%EB%8D%B0%EB%B0%B1%ED%99%94%EC%A0%90' 
import json
import ssl

# def getRequestUrl(url):
#     req= Request(url)

#     try :
#         res = urlopen(req)
#         if res.getcode() == 200:
#             return res.read().decode('utf-8')
#     except  Exception as e:
#         print(e)
#         return None

# api_url+='?serviceKey=4n4Miwzm5p37SLb9Jk9bJa%2FMhFYSTl8mkQIensYxsOuwWyjpePzkk6oyRp3pOsd8GVnzwwQelKHMwSc0bPVfSA%3D%3D'
# api_url+='&pageNo=1'
# api_url+='&numOfRows=10'  
# api_url+=f'&resultType={type}'
# api_url+=f'&stationLoc={quote(stationName)}'

# response = getRequestUrl(api_url)

# if response == None :
#     return None
# else:
#     return json.loads(response)

# quote('롯데백화점')

#stationName 정류소 이름 
def getDataPortalSearch(stationName, type):
    api_url ='https://apis.data.go.kr/6260000/BusanTblBusinfoeqStusService/getTblBusinfoeqStusInfo'
    queryString ="?" +urlencode( 
        {
            'serviceKey': 'SYI3GcjOpXFIvLtehgMOj0BG+gRCuX5NVE6W+LR4jdSXzF2qghYhHBOjZ+jKXu49BxRiWoEW2lkso7szt749/w==',
            'pageNO': '1',
            'numOfRows':'10',
            'resultType':type,
            'stationLoc': stationName
        }
    )
    total_url= api_url +queryString
    response = requests.get(total_url, verify = False)
    return response.text

try:
    result = getDataPortalSearch('부경대', 'json')
    json_data = json.loads(result)
    station_data = json_data['getTblBusinfoeqStusInfo']['body']['items']['item']

    for item in station_data:
        print(item)
except Exception as e:
    print('찾는 데이터가 없습니다.')

{'stationNum': '07006', 'stationLoc': '경성대.부경대역', 'lat': '35.13856124', 'lng': '129.1023074', 'addr': '남구 대연3동 93-7', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '07007', 'stationLoc': '경성대.부경대역', 'lat': '35.13837971', 'lng': '129.1024864', 'addr': '남구 대연3동 90-2', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '07011', 'stationLoc': '부경대대연캠퍼스', 'lat': '35.13287689', 'lng': '129.1012804', 'addr': '남구 대연3동 531-2', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '07012', 'stationLoc': '부경대대연캠퍼스', 'lat': '35.13234515', 'lng': '129.1010586', 'addr': '남구 대연3동 537-1', 'insYear': '', 'dataDay': '2023-02-03'}


c:\Source\studyPython2023\venv\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apis.data.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


### OpenAPI로 가져온 데이터 지도 표시

```python
stop_str = '<h4>'+ item['stationLoc'] +'</h4>' + item['stationNum'] + '<br>' + item['addr']
```

- 문제점
    - 딕셔너리 구조하고 문자열 포맷팅 f'{}' 호환 안됨! -> 문자열 결합 방식 해결

In [24]:
import folium

if len(station_data) > 0:   # 정류소 중 제일 첫번째 인덱스 정류소 위경도를 중심으로
    center_lat = station_data[0]['lat']
    center_lng = station_data[0]['lng']  

    print(center_lat, center_lng)
    print('데이터 존재')

m = folium.Map(location=[center_lat, center_lng], zoom_start=13)

# 전체 정류소 위치값 마커 표시
for item in station_data:
    stop_str = '<h4>'+ item['stationLoc'] +'</h4>' + item['stationNum'] + '<br>' + item['addr']
    iframe = folium.IFrame(stop_str)
    temp = folium.Popup(iframe, min_width=200, max_width=200)  #팝업사이즈 지정

    folium.Marker(location=[item['lat'], item['lng']], popup=temp, icon=folium.Icon(icon='pushpin')).add_to(m)

m

35.13856124 129.1023074
데이터 존재


### BeautifulSoup (version 4)

웹크롤링을 편하게 해주는 도구(라이브러리)

```
pip install beautifulsoup4
```

In [26]:
!pip install beautifulsoup4

### 모듈 import

```python
from bs4 import BeautilfulSoup
```



In [37]:
from bs4 import BeautifulSoup
import requests

url = 'https://kin.naver.com/search/list.naver?query=%EC%A0%9C%EC%9E%84%EC%8A%A4+%EC%9B%B9+%EB%A7%9D%EC%9B%90%EA%B2%BD'

res = requests.get(url)

if res.status_code == 200:  # webpage OK
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    # # print(soup)
    # title = soup.select_one('#s_content > div.section > ul > li:nth-child(1) > dl > dt > a')
    # print(title.get_text())
    ul = soup.select_one('ul.basic1')               # html 소스 순서대로 들어감.
    titles = ul.select('li > dl > dt > a')     
    for title in titles:
        print(title.get_text())

else:
    print(f'Error : {res.status_code}')



제임스웹 우주망원경 질문이요
최고고도 부탁드립니다. 제임스웹 망원경이
제임스 웹 망원경 우주사진 원본 어디서... 
제임스 웹 망원경 촬영 원리
제임스 웹 망원경
제임스 웹 우주망원경이 찍은 첫 사진은... 
제임스 웹 망원경의 종류 알려주세요
제임스 웹 우주망원경
제임스웹 망원경
제임스 웹 우주 망원경 제작 기간이
